<a href="https://colab.research.google.com/github/Mathavk1606/Real-Estate-Demand-Prediction/blob/main/Real_Estate_Demand_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Real Estate Demand Prediction**

## **📋 Table of Contents**

Introduction & Problem Statement

Dataset Overview

Exploratory Data Analysis

Feature Engineering

Model Development

Results & Insights

Conclusions

## **🎯 Introduction & Problem Statement**
In China’s fast-evolving and highly dynamic housing market, accurately forecasting residential demand is vital for investment and development decisions. This competition challenges you to develop a machine learning model that predicts each sector's monthly sales for newly launched private residential projects, using historical transaction data, market conditions, and other relevant features.

In [1]:
!pip install polars
!pip install xgboost
!pip install scikit-optimize
!pip install plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 3.1 MB/s eta 0:00:00


In [4]:
# Core Libraries
import numpy as np
import pandas as pd
import warnings
import os
import polars as pl
import polars.selectors as cs
import calendar
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Styling
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
COLORS = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D', '#6A994E']

# Statistical Analysis
from scipy import stats
from scipy.stats import chi2_contingency, pearsonr, spearmanr, kendalltau, probplot

#sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import xgboost as xgb
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from sklearn.model_selection import TimeSeriesSplit

print("✅ All libraries imported successfully!")
print(f"📦 Pandas version: {pd.__version__}")
print(f"📦 NumPy version: {np.__version__}")
print(f"📦 Polars version: {pl.__version__}")

✅ All libraries imported successfully!
📦 Pandas version: 2.2.2
📦 NumPy version: 2.0.2
📦 Polars version: 1.31.0


In [5]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


# **Group all tables into one for train.csv**

In [6]:
os.chdir('/content/drive/MyDrive/china-real-estate-demand-prediction/train')
base_path = os.getcwd()

new_house_transactions = pd.read_csv(os.path.join(base_path, 'new_house_transactions.csv'))
new_house_transactions_nearby_sectors = pd.read_csv(os.path.join(base_path, 'new_house_transactions_nearby_sectors.csv'))
pre_owned_house_transactions = pd.read_csv(os.path.join(base_path, 'pre_owned_house_transactions.csv'))
pre_owned_house_transactions_nearby_sectors = pd.read_csv(os.path.join(base_path, 'pre_owned_house_transactions_nearby_sectors.csv'))
city_search_index = pd.read_csv(os.path.join(base_path, 'city_search_index.csv'))
sector_POI = pd.read_csv(os.path.join(base_path, 'sector_POI.csv'))
land_transactions = pd.read_csv(os.path.join(base_path, 'land_transactions.csv'))
land_transactions_nearby_sectors = pd.read_csv(os.path.join(base_path, 'land_transactions_nearby_sectors.csv'))
city_indexes = pd.read_csv(os.path.join(base_path, 'city_indexes.csv'))

print("✓ All datasets loaded successfully")

✓ All datasets loaded successfully


In [7]:
#variables

month_dict = {
    "Jan": 1,
    "Feb": 2,
    "Mar": 3,
    "Apr": 4,
    "May": 5,
    "Jun": 6,
    "Jul": 7,
    "Aug": 8,
    "Sep": 9,
    "Oct": 10,
    "Nov": 11,
    "Dec": 12
}

pre_owned_houses = [
    "area_pre_owned_house_transactions",
    "amount_pre_owned_house_transactions",
    "num_pre_owned_house_transactions",
    "price_pre_owned_house_transactions",
    "num_pre_owned_house_transactions_nearby_sectors",
    "area_pre_owned_house_transactions_nearby_sectors",
    "amount_pre_owned_house_transactions_nearby_sectors",
    "price_pre_owned_house_transactions_nearby_sectors"
]

new_houses = [
  "num_new_house_transactions",
  "area_new_house_transactions",
  "price_new_house_transactions",
  "amount_new_house_transactions",
  "area_per_unit_new_house_transactions",
  "total_price_per_unit_new_house_transactions",
  "num_new_house_available_for_sale",
  "area_new_house_available_for_sale",
  "period_new_house_sell_through",
  "num_new_house_transactions_nearby_sectors",
  "area_new_house_transactions_nearby_sectors",
  "price_new_house_transactions_nearby_sectors",
  "amount_new_house_transactions_nearby_sectors",
  "area_per_unit_new_house_transactions_nearby_sectors",
  "total_price_per_unit_new_house_transactions_nearby_sectors",
  "num_new_house_available_for_sale_nearby_sectors",
  "area_new_house_available_for_sale_nearby_sectors",
  "period_new_house_sell_through_nearby_sectors"
]

In [8]:
data = (
    pl.DataFrame(new_house_transactions)
    .join(
        pl.DataFrame(new_house_transactions_nearby_sectors),
        on=['sector','month'],
        how='left',
        suffix='_new_house_transactions_nearby_sectors'
    )
    .join(
        pl.DataFrame(pre_owned_house_transactions),
        on=['sector','month'],
        how='left',
        suffix='_pre_owned_house_transactions'
    )
    .join(
        pl.DataFrame(pre_owned_house_transactions_nearby_sectors),
        on=['sector','month'],
        how='left',
        suffix='_pre_owned_house_transactions_nearby_sectors'
    )
    .join(
        pl.DataFrame(sector_POI),
        on=['sector'],
        how='left',
        suffix='_sector_POI'
    )
    .join(
        pl.DataFrame(land_transactions),
        on=['sector','month'],
        how='left',
        suffix='_land_transactions'
    )
    .join(
        pl.DataFrame(land_transactions_nearby_sectors),
        on=['sector','month'],
        how='left',
        suffix='_land_transactions_nearby_sectors'
    )
    .join(
        pl.DataFrame(city_search_index),
        on=['month'],
        how='left',
        suffix='_city_indexes'
    )
    .with_columns(
        pl.col('month').str.split('-').list.get(0).cast(pl.Int64).alias('Year'),
        pl.col('month').str.split('-').list.get(1).replace(month_dict).cast(pl.Int64).alias('Month'),
        pl.col('sector').str.replace('sector ', '').cast(pl.Int64)
    )
    .drop(['month'])
    .sort(['Year','Month','sector'])
    .fill_nan(0)
    .fill_null(0)
    .with_columns([
        pl.col("amount_new_house_transactions").shift(1).over("sector").alias("amount_lag_1m"),
        pl.col("amount_new_house_transactions").shift(3).over("sector").alias("amount_lag_3m"),
        pl.col("amount_new_house_transactions").shift(6).over("sector").alias("amount_lag_6m"),

        pl.col("amount_new_house_transactions").rolling_mean(3).over("sector").alias("amount_ma_3m"),
        pl.col("amount_new_house_transactions").rolling_mean(6).over("sector").alias("amount_ma_6m"),

        pl.col("amount_new_house_transactions").rolling_std(3).over("sector").alias("amount_std_3m"),
        (pl.col("amount_new_house_transactions") /
         pl.col("amount_new_house_transactions").shift(1).over("sector") - 1).alias("growth_1m"),
        (pl.col("amount_new_house_transactions") /
         pl.col("amount_new_house_transactions").shift(3).over("sector") - 1).alias("growth_3m"),
        pl.col("Month").alias("month_sin").map_elements(lambda x: np.sin(2 * np.pi * x / 12), return_dtype=pl.Float64),
        pl.col("Month").alias("month_cos").map_elements(lambda x: np.cos(2 * np.pi * x / 12), return_dtype=pl.Float64),
        pl.col("Year").alias("year_sin").map_elements(lambda x: np.sin(2 * np.pi * x / 12), return_dtype=pl.Float64),
        pl.col("amount_new_house_transactions").shift(12).over("sector").alias("amount_lag_12m"),
        (pl.col("amount_new_house_transactions") /
         pl.col("amount_new_house_transactions").shift(12).over("sector") - 1).alias("yoy_growth"),
    ])
)

In [9]:
unique_Sectors = data.select('sector').unique().to_pandas()
unique_Year = data.select('Year').unique().to_pandas()
unique_Months = data.select('Month').unique().to_pandas()

cross_all_df = []
for sector in unique_Sectors['sector']:
    for year in unique_Year['Year']:
        for month in unique_Months['Month']:
            cross_all_df.append({
                'sector': sector,
                'Year': year,
                'Month': month
            })
cross_all_df = pl.DataFrame(cross_all_df)

data = (
    data
    .join(
        cross_all_df,
        on=['sector','Year','Month'],
        how='left',
        suffix='_cross_all'
    )
    .fill_nan(0)
)

In [10]:
data.describe()

statistic,sector,num_new_house_transactions,area_new_house_transactions,price_new_house_transactions,amount_new_house_transactions,area_per_unit_new_house_transactions,total_price_per_unit_new_house_transactions,num_new_house_available_for_sale,area_new_house_available_for_sale,period_new_house_sell_through,num_new_house_transactions_nearby_sectors,area_new_house_transactions_nearby_sectors,price_new_house_transactions_nearby_sectors,amount_new_house_transactions_nearby_sectors,area_per_unit_new_house_transactions_nearby_sectors,total_price_per_unit_new_house_transactions_nearby_sectors,num_new_house_available_for_sale_nearby_sectors,area_new_house_available_for_sale_nearby_sectors,period_new_house_sell_through_nearby_sectors,area_pre_owned_house_transactions,amount_pre_owned_house_transactions,num_pre_owned_house_transactions,price_pre_owned_house_transactions,num_pre_owned_house_transactions_nearby_sectors,area_pre_owned_house_transactions_nearby_sectors,amount_pre_owned_house_transactions_nearby_sectors,price_pre_owned_house_transactions_nearby_sectors,sector_coverage,population_scale,residential_area,office_building,commercial_area,resident_population,office_population,number_of_shops,catering,…,medical_health_pharmaceutical_healthcare_dense,medical_health_rehabilitation_institution_dense,medical_health_first_aid_center_dense,medical_health_blood_donation_station_dense,medical_health_disease_prevention_institution_dense,medical_health_general_hospital_dense,medical_health_clinic_dense,education_training_school_education_middle_school_dense,education_training_school_education_primary_school_dense,education_training_school_education_kindergarten_dense,education_training_school_education_research_institution_dense,num_land_transactions,construction_area,planned_building_area,transaction_amount,num_land_transactions_nearby_sectors,construction_area_nearby_sectors,planned_building_area_nearby_sectors,transaction_amount_nearby_sectors,keyword,source,search_volume,Year,Month,amount_lag_1m,amount_lag_3m,amount_lag_6m,amount_ma_3m,amount_ma_6m,amount_std_3m,growth_1m,growth_3m,month_sin,month_cos,year_sin,amount_lag_12m,yoy_growth
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,…,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,325980.0,"""325980""","""325980""",325980.0,325980.0,325980.0,325885.0,325695.0,325410.0,325790.0,325505.0,325790.0,325885.0,325695.0,325980.0,325980.0,325980.0,324840.0,324840.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""0""","""0""",0.0,0.0,0.0,95.0,285.0,570.0,190.0,475.0,190.0,95.0,285.0,0.0,0.0,0.0,1140.0,1140.0
"""mean""",47.106755,89.184428,9648.461992,43986.20081,32587.559558,128.622492,618.834116,1094.54942,126793.684705,21.709812,80.62295,8651.395456,35834.095438,27286.857314,99.733962,420.039071,1071.083526,123442.423958,18.584628,6284.543346,17104.298078,67.450764,24360.805506,69.200273,6432.296763,17769.255815,24779.637937,0.556075,1.7884e6,1300.227315,656.344929,132.289159,1.3194e6,778810.789803,49135.130131,14428.700534,…,0.000298,0.000073,2.9990e-7,7.1960e-7,0.000003,0.00002,0.000069,0.000018,0.000027,0.000061,0

In [11]:
class UsefullFunctions:
    def __init__(self, df):
        self.df = df
    def duplicate_checker(self):
      duplicate_cols = pd.Series(data.columns).duplicated().tolist()

      print(f"\n DUPLICATE COLUMNS")
      if not any(duplicate_cols):
          print(f"    No duplicate column names found")
      else:
          print(f"    {sum(duplicate_cols)} duplicate column names detected:")
          print(f"   Duplicates: {pd.Series(data.columns)[pd.Series(data.columns).duplicated()].tolist()}")

      duplicates = data.is_duplicated().sum()
      print(f"\n DUPLICATE ROWS")
      if duplicates == 0:
          print(f"    No duplicate records found")
      else:
          print(f"    {duplicates} duplicate rows detected")

In [12]:
fig, axs = plt.subplots(figsize=(20, 15))

axs.scatter(data["sector"], data["price_pre_owned_house_transactions"],
           alpha=0.3, s=10, label='Pre-owned House Prices')
axs.scatter(data["sector"], data["price_new_house_transactions"],
           alpha=0.3, s=10, label='New House Prices')

axs.set_xlabel('Sector')
axs.set_ylabel('Price')
axs.set_title('House Prices by Sector')
axs.legend()
axs.grid(True, alpha=0.3)

axs.xaxis.set_major_locator(plt.MaxNLocator(nbins=32, integer=True))
axs.yaxis.set_major_locator(plt.MaxNLocator(nbins=35, integer=True))

plot_dir = '/content/drive/MyDrive/china-real-estate-demand-prediction/diagnostic_plots'
x_label = axs.get_xlabel().replace(" ", "_").replace(" ", "-")
fig_path = os.path.join(plot_dir, f'scatter_plot_{x_label}.png')
plt.savefig(fig_path, dpi=300, bbox_inches='tight')
plt.close(fig)


In [13]:
fig, axs = plt.subplots(figsize=(20, 15))

axs.scatter(data["num_new_house_transactions"],data["sector"],
           alpha=0.3, s=10, label='Pre-owned Houses')
axs.scatter(data["num_pre_owned_house_transactions"],data["sector"],
           alpha=0.3, s=10, label='New Houses')

axs.set_xlabel('No of houses')
axs.set_ylabel('Sector')
axs.set_title('Houses by Sector')
axs.legend()
axs.grid(True, alpha=0.3)

axs.xaxis.set_major_locator(plt.MaxNLocator(nbins=15, integer=True))
axs.yaxis.set_major_locator(plt.MaxNLocator(nbins=32, integer=True))

plt.tight_layout()
plot_dir = '/content/drive/MyDrive/china-real-estate-demand-prediction/diagnostic_plots'
x_label = axs.get_xlabel().replace(" ", "_").replace(" ", "-")
fig_path = os.path.join(plot_dir, f'scatter_plot_{x_label}.png')
plt.savefig(fig_path, dpi=300, bbox_inches='tight')
plt.close(fig)

In [14]:
class CollinearityAnalysis:
    def __init__(self, df):
        self.df = df.to_pandas()
        self.plots_dir = '/content/drive/MyDrive/china-real-estate-demand-prediction/diagnostic_plots'
        os.makedirs(self.plots_dir, exist_ok=True)
        self.qq_plots_dir = os.path.join(self.plots_dir, 'qq_plots')
        self.box_plots_dir = os.path.join(self.plots_dir, 'box_plots')
        os.makedirs(self.qq_plots_dir, exist_ok=True)
        os.makedirs(self.box_plots_dir, exist_ok=True)

    def normality_test(self):
        normality_results = {}
        for col in self.df.columns:
            filtered_data = self.df[col][self.df[col] != -1]
            clean_data = filtered_data.dropna()

            if len(clean_data) < 3:
                normality_results[col] = {
                    'statistic': np.nan,
                    'p_value': np.nan,
                    'is_normal': False
                }
            else:
                try:
                    stat, p_val = stats.shapiro(clean_data)
                    normality_results[col] = {
                        'statistic': stat,
                        'p_value': p_val,
                        'is_normal': p_val > 0.05
                    }
                except ValueError as e:
                    print(f"Warning: Shapiro-Wilk test failed for column '{col}': {e}. Setting as non-normal.")
                    normality_results[col] = {
                        'statistic': np.nan,
                        'p_value': np.nan,
                        'is_normal': False
                    }
                except Exception as e:
                    print(f"Warning: An unexpected error occurred during Shapiro-Wilk test for column '{col}': {e}. Setting as non-normal.")
                    normality_results[col] = {
                        'statistic': np.nan,
                        'p_value': np.nan,
                        'is_normal': False
                    }
        self.normality_results = normality_results

    def plot_qq(self):
        if not hasattr(self, 'normality_results'):
            self.normality_test()

        qq_results = {}
        for col in self.df.columns:
            filtered_data = self.df[col][self.df[col] != -1]
            clean_data = filtered_data.dropna()

            if len(clean_data) < 3:
                qq_results[col] = {'figure_path': None}
                continue

            fig, ax = plt.subplots(figsize=(8, 6))
            probplot(clean_data, dist="norm", plot=ax)
            ax.set_title(f'QQ Plot for {col}')
            ax.set_xlabel('Theoretical Quantiles')
            ax.set_ylabel('Sample Quantiles')

            fig_path = os.path.join(self.qq_plots_dir, f'qq_plot_{col}.png')
            plt.savefig(fig_path, dpi=300, bbox_inches='tight')
            plt.close(fig)

            qq_results[col] = {
                'figure_path': fig_path,
                'is_normal': self.normality_results.get(col, {}).get('is_normal', False)
            }

        self.qq_results = qq_results
        print(f"QQ plots saved in '{self.qq_plots_dir}' directory.")

    def plot_boxplots(self):
        box_results = {}
        for col in self.df.columns:
            filtered_data = self.df[col][self.df[col] != -1]
            clean_data = filtered_data.dropna()

            if len(clean_data) < 4:
                box_results[col] = {'figure_path': None}
                continue

            fig, ax = plt.subplots(figsize=(8, 6))
            ax.boxplot(clean_data)
            ax.set_title(f'Boxplot for {col}')
            ax.set_ylabel(col)

            fig_path = os.path.join(self.box_plots_dir, f'boxplot_{col}.png')
            plt.savefig(fig_path, dpi=300, bbox_inches='tight')
            plt.close(fig)

            box_results[col] = {
                'figure_path': fig_path
            }

        self.box_results = box_results
        print(f"Boxplots saved in '{self.box_plots_dir}' directory.")

    def ordinal_test(self):
        ordinal_results = {}
        for col in self.df.columns:
            filtered_data = self.df[col][self.df[col] != -1]
            clean_data = filtered_data.dropna()

            if len(clean_data) < 2:
                ordinal_results[col] = {
                    'is_ordinal': False,
                    'n_unique': 0
                }
                continue

            uniques = clean_data.unique()
            n_unique = len(uniques)

            is_int_like = (clean_data.apply(lambda x: x == int(x) if pd.notnull(x) else False)).all()

            is_ordinal = is_int_like and 2 <= n_unique <= 10

            ordinal_results[col] = {
                'is_ordinal': is_ordinal,
                'n_unique': n_unique
            }
        self.ordinal_results = ordinal_results

    def correlation_analysis(self):
        if not hasattr(self, 'normality_results'):
            self.normality_test()
        if not hasattr(self, 'ordinal_results'):
            self.ordinal_test()
        if not hasattr(self, 'box_results'):
            self.plot_boxplots()

        correlation_results = {}
        columns = self.df.columns.tolist()

        def has_outliers(series, is_normal, n, method='auto'):
            if len(series) < 4:
                return False

            if method == 'z_score':
                mean = series.mean()
                std = series.std()
                if std == 0:
                    return False
                z_scores = np.abs((series - mean) / std)
                return (z_scores > 3).any()
            elif method == 'iqr':
                Q1 = series.quantile(0.25)
                Q3 = series.quantile(0.75)
                IQR = Q3 - Q1
                if IQR == 0:
                    return False
                lower = Q1 - 1.5 * IQR
                upper = Q3 + 1.5 * IQR
                return ((series < lower) | (series > upper)).any()
            else:
                if is_normal and n >= 30:
                    return has_outliers(series, is_normal, n, 'z_score')
                else:
                    return has_outliers(series, is_normal, n, 'iqr')

        for i in range(len(columns)):
            for j in range(i + 1, len(columns)):
                col1 = columns[i]
                col2 = columns[j]

                mask = (self.df[col1] != -1) & (self.df[col2] != -1) & \
                       self.df[col1].notna() & self.df[col2].notna()
                x = self.df.loc[mask, col1]
                y = self.df.loc[mask, col2]

                pair_n = len(x)
                if pair_n < 2:
                    continue

                normal1 = self.normality_results.get(col1, {}).get('is_normal', False)
                normal2 = self.normality_results.get(col2, {}).get('is_normal', False)
                both_normal = normal1 and normal2

                ordinal1 = self.ordinal_results.get(col1, {}).get('is_ordinal', False)
                ordinal2 = self.ordinal_results.get(col2, {}).get('is_ordinal', False)
                at_least_one_ordinal = ordinal1 or ordinal2

                outliers1 = has_outliers(x, normal1, pair_n)
                outliers2 = has_outliers(y, normal2, pair_n)
                has_outliers_pair = outliers1 or outliers2

                if pair_n < 30:
                    corr_func = kendalltau
                    method = 'kendall'
                elif has_outliers_pair or not both_normal or at_least_one_ordinal:
                    corr_func = spearmanr
                    method = 'spearman'
                else:
                    corr_func = pearsonr
                    method = 'pearson'

                try:
                    r, p = corr_func(x, y)
                except ValueError:
                    continue
                except Exception as e:
                    print(f"Warning: Correlation failed for pair ({col1}, {col2}) with method '{method}': {e}")
                    continue

                key = (col1, col2)
                correlation_results[key] = {
                    'method': method,
                    'correlation': float(r),
                    'p_value': float(p),
                    'n': pair_n,
                    'both_normal': both_normal,
                    'has_outliers': has_outliers_pair,
                    'at_least_one_ordinal': at_least_one_ordinal,
                    'outlier_method_x': 'z_score' if normal1 and pair_n >= 30 else 'iqr',
                    'outlier_method_y': 'z_score' if normal2 and pair_n >= 30 else 'iqr'
                }

        self.correlation_results = correlation_results
        if correlation_results:
            corr_df = pd.DataFrame([
                {'col1': k[0], 'col2': k[1], **v} for k, v in correlation_results.items()
            ])
            return corr_df
        else:
            return pd.DataFrame()


In [15]:
polar_df0 = data.select(pre_owned_houses)
ca0 = CollinearityAnalysis(polar_df0)

ca0.normality_test()
ca0.plot_qq()
ca0.plot_boxplots()
ca0.ordinal_test()

QQ plots saved in '/content/drive/MyDrive/china-real-estate-demand-prediction/diagnostic_plots/qq_plots' directory.
Boxplots saved in '/content/drive/MyDrive/china-real-estate-demand-prediction/diagnostic_plots/box_plots' directory.


In [16]:
polar_df1 = data.select(new_houses)
ca1 = CollinearityAnalysis(polar_df1)

ca1.normality_test()
ca1.plot_qq()
ca1.plot_boxplots()
ca1.ordinal_test()

QQ plots saved in '/content/drive/MyDrive/china-real-estate-demand-prediction/diagnostic_plots/qq_plots' directory.
Boxplots saved in '/content/drive/MyDrive/china-real-estate-demand-prediction/diagnostic_plots/box_plots' directory.


In [17]:
pre_owned_features_corr_df = ca0.correlation_analysis()
new_houses_features_corr_df = ca1.correlation_analysis()

columns = sorted(set(pre_owned_features_corr_df['col1'].tolist() + pre_owned_features_corr_df['col2'].tolist()))
corr = pd.DataFrame(np.eye(len(columns)), index=columns, columns=columns)

for _, row in pre_owned_features_corr_df.iterrows():
    col1, col2 = row['col1'], row['col2']
    corr_val = row['correlation']
    corr.loc[col1, col2] = corr_val
    corr.loc[col2, col1] = corr_val

fig, ax = plt.subplots(figsize=(12, 10))
im = ax.imshow(corr, cmap='coolwarm', aspect='auto', vmin=-1, vmax=1)

ax.set_xticks(np.arange(len(corr.columns)))
ax.set_yticks(np.arange(len(corr.columns)))
ax.set_xticklabels(corr.columns, rotation=45, ha='right')
ax.set_yticklabels(corr.columns)

cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Correlation', rotation=270, labelpad=15)

for i in range(len(corr.columns)):
    for j in range(len(corr.columns)):
        text = ax.text(j, i, f'{corr.iloc[i, j]:.2f}',
                      ha="center", va="center", color="black", fontsize=8)

plt.title('Correlation Heatmap - New Houses')
plt.tight_layout()

plot_dir = '/content/drive/MyDrive/china-real-estate-demand-prediction/diagnostic_plots'
fig_path = os.path.join(plot_dir, f'corr_plot_{ax.title}.png')
plt.savefig(fig_path, dpi=300, bbox_inches='tight')
plt.close(fig)

columns = sorted(set(new_houses_features_corr_df['col1'].tolist() + new_houses_features_corr_df['col2'].tolist()))
corr = pd.DataFrame(np.eye(len(columns)), index=columns, columns=columns)

for _, row in new_houses_features_corr_df.iterrows():
    col1, col2 = row['col1'], row['col2']
    corr_val = row['correlation']
    corr.loc[col1, col2] = corr_val
    corr.loc[col2, col1] = corr_val

fig, ax = plt.subplots(figsize=(12, 10))
im = ax.imshow(corr, cmap='coolwarm', aspect='auto', vmin=-1, vmax=1)

ax.set_xticks(np.arange(len(corr.columns)))
ax.set_yticks(np.arange(len(corr.columns)))
ax.set_xticklabels(corr.columns, rotation=45, ha='right')
ax.set_yticklabels(corr.columns)

cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Correlation', rotation=270, labelpad=15)

for i in range(len(corr.columns)):
    for j in range(len(corr.columns)):
        text = ax.text(j, i, f'{corr.iloc[i, j]:.2f}',
                      ha="center", va="center", color="black", fontsize=8)

plt.title('Correlation Heatmap - New Houses')
plt.tight_layout()

plot_dir = '/content/drive/MyDrive/china-real-estate-demand-prediction/diagnostic_plots'
fig_path = os.path.join(plot_dir, f'corr_plot_{ax.title}.png')
plt.savefig(fig_path, dpi=300, bbox_inches='tight')
plt.close(fig)

In [18]:
variables = sector_POI.columns.to_list() + [
    "sector",
    "Year",
    "price_pre_owned_house_transactions",
    "price_new_house_transactions"
]

variables = list(dict.fromkeys(variables))

new_df = data.select(variables)
aggregated_data = new_df.group_by(['sector', 'Year']).agg([
    pl.col('price_pre_owned_house_transactions').mean().alias('mean_pre_owned'),
    pl.col('price_new_house_transactions').mean().alias('mean_new')
]).sort(['sector', 'Year'])

aggregated_pd = aggregated_data.to_pandas()

sectors = aggregated_pd['sector'].unique()
num_sectors = len(sectors)

ncols = 3
nrows = (num_sectors + ncols - 1) // ncols
fig, axes = plt.subplots(nrows, ncols, figsize=(15, 5 * nrows))
axes = axes.flatten() if num_sectors > 1 else [axes]

for idx, sector in enumerate(sectors):
    ax = axes[idx]
    sector_data = aggregated_pd[aggregated_pd['sector'] == sector]

    ax.plot(sector_data['Year'], sector_data['mean_pre_owned'],
            marker='o', label='Pre-Owned', linewidth=2)
    ax.plot(sector_data['Year'], sector_data['mean_new'],
            marker='s', label='New', linewidth=2, linestyle='--')

    ax.set_xlabel('Year')
    ax.set_ylabel('Mean Transaction Price')
    ax.set_title(f'Sector: {sector}')
    ax.legend()
    ax.grid(True, alpha=0.3)

for idx in range(num_sectors, len(axes)):
    axes[idx].axis('off')

plt.suptitle('Mean House Transaction Prices by Year and Sector', fontsize=16, y=1.00)
plt.tight_layout()

plot_dir = '/content/drive/MyDrive/china-real-estate-demand-prediction/diagnostic_plots'
fig_path = os.path.join(plot_dir, f'line_plot_total.png')
plt.savefig(fig_path, dpi=300, bbox_inches='tight')
plt.close(fig)



In [19]:
def custom_competition_score(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    if y_true.shape != y_pred.shape:
        raise ValueError("y_true and y_pred must have the same shape.")
    ape = np.empty_like(y_true, dtype=float)
    zero_mask = (y_true == 0)
    nonzero_mask = ~zero_mask
    ape[nonzero_mask] = np.abs(y_pred[nonzero_mask] - y_true[nonzero_mask]) / np.abs(y_true[nonzero_mask])
    ape[zero_mask] = np.where(np.abs(y_pred[zero_mask]) == 0, 0.0, np.inf)
    frac_over_1 = np.mean(ape > 1.0)
    if frac_over_1 > 0.30:
        return 0.0
    ok_mask = (ape <= 1.0)
    frac_le_1 = np.mean(ok_mask)
    mape_ok = np.mean(ape[ok_mask])
    scaled_mape = mape_ok / frac_le_1
    score = 1.0 - scaled_mape
    return float(score)

In [20]:
data_train = (
    data
    .filter(
        (pl.col("Year") < 2023) |
        ((pl.col("Year") == 2023) & (pl.col("Month") <= 7))
    )
    .sort(["Year", "Month", "sector"])
    .with_columns(
        pl.col("amount_new_house_transactions")
          .shift(-12)
          .over("sector")
          .alias("target_12m_ahead")
    )
    .fill_null(0)
)

In [21]:
X = data_train.drop("target_12m_ahead")
y = data_train.select("target_12m_ahead")

In [22]:
TARGET_COL = "target_12m_ahead"

X_tr_list, X_val_list, y_tr_list, y_val_list = [], [], [], []

for sector, group in data_train.group_by("sector"):
    group = group.sort(["Year", "Month"])

    X_grp = group.drop(TARGET_COL)
    y_grp = group.select(TARGET_COL)

    n = len(group)
    split_idx = int(n * 0.80)

    X_tr = X_grp[:split_idx]
    X_val = X_grp[split_idx:]
    y_tr = y_grp[:split_idx]
    y_val = y_grp[split_idx:]

    X_tr_list.append(X_tr)
    X_val_list.append(X_val)
    y_tr_list.append(y_tr)
    y_val_list.append(y_val)

X_tr = pl.concat(X_tr_list)
X_val = pl.concat(X_val_list)
y_tr = pl.concat(y_tr_list)
y_val = pl.concat(y_val_list)

print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape)

(212928, 194) (53232, 194) (212928, 1) (53232, 1)


In [23]:
def clean_xy(X, y):
    y_clean = (
        y
        .with_columns(
            pl.col("target_12m_ahead")
            .cast(pl.Float64, strict=False)
            .replace([float('inf'), float('-inf')], None)
        )
    )

    ok_mask = y_clean.select(
        pl.col("target_12m_ahead").is_not_null()
    ).to_series()

    X_clean = X.filter(ok_mask)
    y_clean = y_clean.filter(ok_mask)

    return X_clean, y_clean

X_tr, y_tr = clean_xy(X_tr, y_tr)
X_val, y_val = clean_xy(X_val, y_val)

In [24]:
non_numeric_cols = []
for col in X_tr.columns:
    dtype = X_tr[col].dtype
    if dtype not in [pl.Int8, pl.Int16, pl.Int32, pl.Int64,
                     pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64,
                     pl.Float32, pl.Float64, pl.Boolean]:
        non_numeric_cols.append(col)

print(f"Non-numeric columns to handle: {non_numeric_cols}")

X_tr = X_tr.drop(non_numeric_cols)
X_val = X_val.drop(non_numeric_cols)

X_tr_pd = X_tr.to_pandas()
y_tr_np = y_tr.select("target_12m_ahead").to_numpy().ravel()

X_val_pd = X_val.to_pandas()
y_val_np = y_val.select("target_12m_ahead").to_numpy().ravel()

Non-numeric columns to handle: ['keyword', 'source']


In [26]:
search_spaces = {
    'n_estimators': Integer(1000, 4000),  # Much smaller
    'learning_rate': Real(0.015, 0.04, prior='log-uniform'),
    'max_depth': Integer(5, 7),
    'min_child_weight': Integer(2, 8),
    'subsample': Real(0.85, 0.95, prior='uniform'),
    'colsample_bytree': Real(0.85, 0.95, prior='uniform'),
    'gamma': Real(0, 0.3, prior='uniform'),
    'reg_alpha': Real(0, 0.3, prior='uniform'),
    'reg_lambda': Real(0.8, 1.5, prior='uniform'),
}

tscv = TimeSeriesSplit(n_splits=3)  # Reduced

base_model = xgb.XGBRegressor(
    objective="reg:tweedie",
    tweedie_variance_power=1.4,
    random_state=42,
    tree_method="hist",
    eval_metric="rmse",
    early_stopping_rounds=50,  # More aggressive
    n_estimators=4000,  # Reduced
)

bayes_search = BayesSearchCV(
    estimator=base_model,
    search_spaces=search_spaces,
    n_iter=25,  # Reduced
    cv=tscv,
    scoring='neg_root_mean_squared_error',
    verbose=2,
    random_state=42,
    n_jobs=-1,
)

In [28]:
bayes_search.fit(
    X_tr_pd,
    y_tr_np,
    eval_set=[(X_val_pd, y_val_np)],  # Add validation set
    verbose=False
)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fi

BayesSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
              estimator=XGBRegressor(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=50,
                                     enable_categorical=False,
                                     eval_metric='rmse', feature_types=None,
                                     feature_weights=None, gamma=Non...
                             'min_child_weight': Integer(low=2, high=8, prior='uniform', transform='normalize'),
                             'n_estimators': Integer(low=1000, high=4000, prior='uniform', transform='normalize'),
                             'reg_alpha': Real(low=0, high=0.3, prior='uniform', transform='normalize'),
                             'reg_lambda': Real(low=0.8, high=1.5, prior='uniform', transform='normalize'),
                             'subsample': Real(low=0.85, high=0.95, prior='uniform', transform='normalize')},
              verbose=2)

In [29]:
model = bayes_search.best_estimator_

In [30]:
y_temp = model.predict(X_val_pd)
y_pred = np.asarray(y_temp)
y_true = y_val_np

mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_true, y_pred)
custom = custom_competition_score(y_true, y_pred)

print(f"[VAL] RMSE: {rmse:,.3f} | R^2: {r2:,.4f} | Custom: {custom:,.4f}")

[VAL] RMSE: 13,436.357 | R^2: 0.8681 | Custom: 0.7785


In [31]:
data_test = (
    data
    .filter(
        (pl.col("Year") > 2023) |
        ((pl.col("Year") == 2023) & (pl.col("Month") > 7))
    )
)

In [32]:
data_test = data_test.select(cs.numeric())
data_test_pd = data_test.to_pandas()


In [33]:
test_result = model.predict(data_test_pd)

In [34]:
data_test = data_test.with_columns(
    pl.lit(test_result).alias("target_predicted")
)

In [35]:
mabbr = {i: calendar.month_abbr[i] for i in range(1, 13)}

map_df = (
    data_test
    .select(["Year", "Month", "sector", "target_predicted"])
    .with_columns(
        (
            (pl.col("Year") + 1).cast(pl.Utf8) + " " +
            pl.col("Month").map_elements(lambda x: mabbr.get(x, ""), return_dtype=pl.Utf8) + "_sector " +
            pl.col("sector").cast(pl.Utf8)
        ).alias("id")
    )
    .group_by("id", maintain_order=True)
    .agg(pl.col("target_predicted").last())
)

In [36]:
comp_test_df = pl.read_csv("/content/drive/MyDrive/china-real-estate-demand-prediction/test.csv")
comp_test_df = comp_test_df.with_columns(
    pl.col("new_house_transaction_amount").cast(pl.Float64, strict=False)
)

In [37]:
comp_test_df = (
    comp_test_df
    .join(map_df, on="id", how="left")
    .with_columns(
        pl.when(pl.col("new_house_transaction_amount").is_null())
        .then(pl.col("target_predicted"))
        .otherwise(pl.col("new_house_transaction_amount"))
        .alias("new_house_transaction_amount")
    )
    .drop("target_predicted")
)

In [38]:
comp_test_df = comp_test_df.fill_null(0)

In [39]:
comp_test_df.write_csv("/content/drive/MyDrive/china-real-estate-demand-prediction/submission.csv")